In [ ]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
from skimage.transform import resize
from tensorflow import keras
from scipy.spatial import distance

def detect_faces(frame):
    face = detector(frame)
    faces = []
    for i in range(len(face)):
        x = face[i].left()
        y = face[i].top()
        w = face[i].right()-face[i].left()
        h = face[i].bottom()-face[i].top()
        if x<0:
            x = 0
        if y<0:
            y = 0
        faces.append((x, y, w, h))
    return faces

#圖像的預處理(即前述的幾項步驟)
def preProcess(img):
    whitenImg = prewhiten(img)
    whitenImg = whitenImg[np.newaxis, :]
    return whitenImg

#圖像白化（whitening）可用於對過度曝光或低曝光的圖片進行處理，處理的方式就是改變圖像的平均像素值為 0 ，改變圖像的方差為單位方差 1。
def prewhiten(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError("Dimension should be 3 or 4")
    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

# 匯入模型
model = keras.models.load_model("facenet_keras.h5", compile=False)
detector = dlib.get_frontal_face_detector()

# 設定初始參數
image_size = 160

# 資料庫中的人臉資料
img = cv2.imread("Image_test/Chieh-Ming Yang.jpeg")
faces = detect_faces(img)
(x, y, w, h) = faces[0]
face = img[y:y+h, x:x+w]
aligned = resize(face, (image_size, image_size), mode="reflect")
faceImg = preProcess(aligned)
embs_valid_Yang = l2_normalize(np.concatenate(model.predict(faceImg)))

# 資料庫中的人臉資料
img = cv2.imread("Image_test/Jen-Yeu Chen.png")
faces = detect_faces(img)
(x, y, w, h) = faces[0]
face = img[y:y+h, x:x+w]
aligned = resize(face, (image_size, image_size), mode="reflect")
faceImg = preProcess(aligned)
embs_valid_Chen = l2_normalize(np.concatenate(model.predict(faceImg)))

# 開啟攝影機
VIDEO_IN = cv2.VideoCapture(0)

while True:
    hasFrame, img = VIDEO_IN.read()
    img = cv2.resize(img, None, fx=0.5, fy=0.5)
    
    faces = detect_faces(img)
    if(faces != []):
        
        for faces in faces:
            (x, y, w, h) = faces
            face = img[y:y+h, x:x+w]
            aligned = resize(face, (image_size, image_size), mode="reflect")
            faceImg = preProcess(aligned)
            embs = l2_normalize(np.concatenate(model.predict(faceImg)))

            # 計算兩個人臉特徵的距離
            distanceNum_Yang = distance.euclidean(embs_valid_Yang, embs)
            distanceNum_Chen = distance.euclidean(embs_valid_Chen, embs)

            if distanceNum_Yang <0.9:
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)
                cv2.putText(img,"{}: {}".format("Chieh-Ming Yang",round(distanceNum_Yang,2)),(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            
            elif distanceNum_Chen <0.9:
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255),2)
                cv2.putText(img,"{}: {}".format("Jen-Yeu Chen",round(distanceNum_Chen,2)),(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
            
            else:
                cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0),2)
                cv2.putText(img,"{}".format("Other"),(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)
        
    cv2.imshow("Frame", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
VIDEO_IN.release()
cv2.destroyAllWindows()